<a href="https://colab.research.google.com/github/tanvircr7/Alympics/blob/main/tool_call_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install required packages
!pip install -q torch tqdm datasets jsonlines matplotlib seaborn pandas numpy scikit-learn

# Import core libraries
import torch
import json
import time
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from typing import Dict, List, Tuple, Any, Optional, Union

print("✅ Environment setup complete")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA device count: {torch.cuda.device_count()}")
if torch.cuda.is_available():
    print(f"CUDA device name: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

✅ Environment setup complete
PyTorch version: 2.9.1+cu128
CUDA available: True
CUDA device count: 1
CUDA device name: NVIDIA L4
GPU memory: 23.80 GB


In [ ]:
# Trigger runtime restart after upgrade
import os
import sys

print("🔁 Restarting runtime to apply bitsandbytes upgrade...")

# Upgrade bitsandbytes first
!pip install -U "bitsandbytes>=0.46.1" "accelerate>=0.30.0" "transformers>=4.39.0"

print("✅ Packages upgraded. Restarting...")
os.execv(sys.executable, ['python'] + sys.argv)

🔁 Restarting runtime to apply bitsandbytes upgrade...


In [ ]:
# 🔧 Force upgrade to latest stable versions
!pip install -U \
  "transformers>=4.44.0" \
  "accelerate>=0.32.0" \
  "bitsandbytes>=0.46.1" \
  "torch>=2.3.0" \
  "safetensors>=0.4.0"

print("✅ Packages upgraded. Now restarting runtime to apply changes.")
import os, sys
os.execv(sys.executable, ['python'] + sys.argv)

In [1]:
import transformers, accelerate, bitsandbytes, torch
print("✅ Versions:")
print(f"transformers: {transformers.__version__} (≥4.44.0 ✅)")
print(f"accelerate:   {accelerate.__version__} (≥0.32.0 ✅)")
print(f"bitsandbytes: {bitsandbytes.__version__} (≥0.46.1 ✅)")
print(f"PyTorch:      {torch.__version__}")

# Optional: suppress future deprecation warning
import warnings
warnings.filterwarnings("ignore", message="`torch_dtype` is deprecated! Use `dtype` instead!")

# Now load model
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
print("📥 Tokenizer loaded.")

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.float16
)

print("✅ Model loaded successfully!")
print(f"Device: {model.device}")
print(f"Memory footprint: {model.get_memory_footprint() / 1e9:.2f} GB")

✅ Versions:
transformers: 5.0.0.dev0 (≥4.44.0 ✅)
accelerate:   1.13.0.dev0 (≥0.32.0 ✅)
bitsandbytes: 0.48.2 (≥0.46.1 ✅)
PyTorch:      2.9.1+cu128


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


📥 Tokenizer loaded.


`torch_dtype` is deprecated! Use `dtype` instead!


Loading weights:   0%|          | 0/339 [00:00<?, ?it/s]

Qwen2ForCausalLM LOAD REPORT from: Qwen/Qwen2.5-7B-Instruct
Key                                                                         | Status  | 
----------------------------------------------------------------------------+---------+-
model.layers.{0...27}.mlp.up_proj.weight.quant_state.bitsandbytes__nf4      | MISSING | 
model.layers.{0...27}.self_attn.k_proj.weight.nested_absmax                 | MISSING | 
model.layers.{0...27}.self_attn.v_proj.weight                               | MISSING | 
model.layers.{0...27}.mlp.gate_proj.weight.nested_quant_map                 | MISSING | 
model.layers.{0...27}.mlp.down_proj.weight                                  | MISSING | 
model.layers.{0...27}.self_attn.v_proj.weight.absmax                        | MISSING | 
model.layers.{0...27}.mlp.up_proj.weight.absmax                             | MISSING | 
model.layers.{0...27}.self_attn.k_proj.weight                               | MISSING | 
model.layers.{0...27}.self_attn.k_proj.weight.quan

NotImplementedError: Cannot copy out of meta tensor; no data! Please use torch.nn.Module.to_empty() instead of torch.nn.Module.to() when moving module from meta to a different device.

In [1]:
# ✅ CELL 1: Environment setup (post-restart)
import torch
import warnings
warnings.filterwarnings("ignore")

print("🔧 Environment check:")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

# ✅ CELL 2: Load Qwen2.5-7B-Instruct with 4-bit
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    BitsAndBytesConfig, pipeline
)

MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"

print(f"\n📥 Loading {MODEL_NAME} with 4-bit quantization...")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.float16
)

generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    max_new_tokens=256,
    temperature=0.3,
    top_p=0.95,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id
)

print("✅ Model loaded successfully!")
print(f"Model device: {model.device}")
print(f"Quantization type: 4-bit (NF4)")

🔧 Environment check:
PyTorch: 2.9.1+cu128
CUDA available: True
GPU: NVIDIA L4

📥 Loading Qwen/Qwen2.5-7B-Instruct with 4-bit quantization...


`torch_dtype` is deprecated! Use `dtype` instead!


Loading weights:   0%|          | 0/339 [00:00<?, ?it/s]

Qwen2ForCausalLM LOAD REPORT from: Qwen/Qwen2.5-7B-Instruct
Key                                                                         | Status  | 
----------------------------------------------------------------------------+---------+-
model.layers.{0...27}.mlp.gate_proj.weight.quant_map                        | MISSING | 
model.layers.{0...27}.self_attn.k_proj.weight.nested_absmax                 | MISSING | 
model.layers.{0...27}.self_attn.v_proj.weight.nested_quant_map              | MISSING | 
model.layers.{0...27}.self_attn.k_proj.weight                               | MISSING | 
model.layers.{0...27}.mlp.gate_proj.weight.absmax                           | MISSING | 
model.layers.{0...27}.mlp.down_proj.weight.quant_map                        | MISSING | 
model.layers.{0...27}.mlp.up_proj.weight.quant_map                          | MISSING | 
model.layers.{0...27}.self_attn.v_proj.weight                               | MISSING | 
model.layers.{0...27}.self_attn.v_proj.weight.quan

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

NotImplementedError: Cannot copy out of meta tensor; no data! Please use torch.nn.Module.to_empty() instead of torch.nn.Module.to() when moving module from meta to a different device.